In [ ]:
#pip install pandas cohere datasets python-dotenv annoy matplotlib numpy

In [ ]:
#pip install

In [1]:
from datasets import load_dataset
import pandas as pd

# טעינת המאגר מהאינטרנט (כפי שמופיע בקוד של המורה)
dataset = load_dataset("MaBrukAI/ai-questions-hebrew", split="train")
df = dataset.to_pandas()

# נסתכל על 10 השורות הראשונות כדי לוודא שזה עבד
df.head(10)

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,text
0,1,מה ההבדל בין למידת מכונה ללמידה עמוקה?
1,2,איך מאמנים מודל סיווג טקסטים באמצעות Scikit-le...
2,3,מה היתרונות של שימוש ב‑transformers לעומת RNNs?
3,4,איך משתמשים ב‑TensorFlow לבניית רשת עצבית רבת ...
4,5,מה זה embedding ואיך משתמשים בזה בעיבוד שפה טב...
5,6,איך לאמן מודל זיהוי רגשות בטקסט בעברית?
6,7,מה ההבדל בין supervised ו‑unsupervised learning?
7,8,מה היתרונות והחסרונות של שימוש במודל GPT בפרוי...
8,9,איך אפשר לבצע fine-tuning למודל BERT בעברית?
9,10,איך ניתן לייעל inference של מודלי שפה בג'אווה ...


In [2]:
import cohere
import os
from dotenv import load_dotenv

# טעינת המפתח מהקובץ
load_dotenv()
api_key = os.getenv("COHERE_API_KEY")

# יצירת הלקוח - זה ה-'co' שהיה חסר
co = cohere.Client(api_key)
print("Connected to Cohere!")

Connected to Cohere!


In [3]:
model_name = "embed-multilingual-v3.0"
input_type_embed = "search_document"

# שליחת הטקסטים ל-API של Cohere
embeds = co.embed(
    texts=list(df["text"]),
    model=model_name,
    input_type=input_type_embed,
).embeddings

print(f"נוצרו {len(embeds)} וקטורים בהצלחה!")

נוצרו 100 וקטורים בהצלחה!


In [4]:
from annoy import AnnoyIndex
import numpy as np

# הגדרת האינדקס (שימוש ב-1024 ממדים שזה הסטנדרט של המודל הרב-לשוני)
embed_size = np.array(embeds).shape[1]
search_index = AnnoyIndex(embed_size, "angular")

# הוספת השאלות לאינדקס
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10)
print("Search index is ready!")

Search index is ready!


In [5]:
# הגדרת השאילתה שלך
query = "מה טוב ומה לא טוב ב-ChatGPT?"
input_type_query = "search_query"

# 1. הפיכת השאלה שלך לווקטור (Embedding)
query_embed = co.embed(
    texts=[query], 
    model=model_name, 
    input_type=input_type_query
).embeddings

# 2. שליפת השכנים הקרובים ביותר מהאינדקס שבנינו (Annoy)
# אנחנו מבקשים את 10 התוצאות הכי קרובות וגם את המרחק שלהן (include_distances=True)
similar_item_ids = search_index.get_nns_by_vector(
    query_embed[0], 10, include_distances=True
)

# 3. ארגון התוצאות בטבלה יפה (DataFrame)
query_results = pd.DataFrame(
    data={
        "text": df.iloc[similar_item_ids[0]]["text"].values,
        "distance": similar_item_ids[1],
    }
)

# הצגת התוצאות
print(f"Query: '{query}'\nNearest neighbors:")
query_results

Query: 'מה טוב ומה לא טוב ב-ChatGPT?'
Nearest neighbors:


,text,distance
0,מה היתרונות והחסרונות של שימוש במודל GPT בפרוי...,0.855579
1,מה היתרונות והחסרונות של שימוש במודל GPT בפרוי...,0.855807
2,מה היתרונות והחסרונות של שימוש במודל GPT בפרוי...,0.855807
3,מה היתרונות והחסרונות של שימוש במודל GPT בפרוי...,0.855807
4,מה היתרונות של שימוש ב‑transformers לעומת RNNs?,0.975679
5,מה היתרונות של שימוש ב‑transformers לעומת RNNs?,0.975679
6,מה היתרונות של שימוש ב‑transformers לעומת RNNs?,0.975679
7,מה היתרונות של שימוש ב‑transformers לעומת RNNs?,0.975679
8,מה ההבדל בין huggingface transformers לבין ker...,0.977379
9,מה ההבדל בין huggingface transformers לבין ker...,0.977379
